In [30]:
import tensorflow

In [31]:
import cv2 
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [47]:
# images dir 
IMG_SIZE = 32
images_dir = 'train'
# function that looks at file name to check whether it's a dog or cat in training data
def label_img(img):
    word_label = img.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #  [much cat, no dog]
    if word_label == 'cat': return 1
    #  [no cat, very doggo]
    elif word_label == 'dog': return 0
# creates training data (converts images to grayscale)
def process_data():
    my_data = []
    my_result = []
    # loop through images in training data directory
    for img in tqdm(os.listdir(images_dir)):
        # set the label for the pixel data - either 1 (cat) or 0 (dog)
        label = label_img(img)
        # get training data image
        path = os.path.join(images_dir,img)
        img = cv2.imread(path)
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        #gray = gray.flatten()
        my_data.append(np.array(gray))
        my_result.append(label)
    np.save('my_data.npy', my_data)
    np.save('my_result.npy', my_result)
    return my_data, my_result

In [48]:
X, y = process_data()

100%|██████████| 25000/25000 [01:59<00:00, 209.75it/s]


In [49]:
X = np.array(X)
X = X/255.

In [50]:
# https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/
# Global Centering -  We can test our model both ways - centering before and centering after dividing by 255 (normalization)
mean_X = np.mean(X)
std_X = np.std(X)
X = (X - mean_X)/std_X

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=(3/4), random_state=0)


In [77]:
from tensorflow.keras import layers, models


In [94]:
# https://www.youtube.com/watch?v=7HPwo4wnJeA&t=349s
ann = models.Sequential([
        layers.Flatten(input_shape=(32,32,3)),
        layers.Dense(1000, activation='relu'),
        layers.Dense(300, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(10, activation='relu'),
        layers.Dense(2, activation='softmax')    
    ])

ann.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [95]:
print(np.array(X_train).shape)
print(np.array(y_train).shape)

(18750, 32, 32, 3)
(18750,)


In [96]:
ann.fit(np.array(X_train), np.array(y_train), epochs=10)


Epoch 1/10
586/586 [==============================] - 15s 23ms/step - loss: 0.6637 - accuracy: 0.5956
Epoch 2/10
586/586 [==============================] - 12s 20ms/step - loss: 0.5977 - accuracy: 0.6714
Epoch 3/10
586/586 [==============================] - 12s 20ms/step - loss: 0.5385 - accuracy: 0.7248
Epoch 4/10
586/586 [==============================] - 12s 20ms/step - loss: 0.4783 - accuracy: 0.7698
Epoch 5/10
586/586 [==============================] - 11s 20ms/step - loss: 0.4134 - accuracy: 0.8105
Epoch 6/10
586/586 [==============================] - 12s 20ms/step - loss: 0.3481 - accuracy: 0.8486
Epoch 7/10
586/586 [==============================] - 12s 21ms/step - loss: 0.2963 - accuracy: 0.8753
Epoch 8/10
586/586 [==============================] - 12s 20ms/step - loss: 0.2462 - accuracy: 0.8981
Epoch 9/10
586/586 [==============================] - 12s 21ms/step - loss: 0.2034 - accuracy: 0.9191
Epoch 10/10
586/586 [==============================] - 13s 21ms/step - loss: 0.179

In [99]:
predict_ann = ann.predict(X_test)
predict_ann_one = []
for outcome in predict_ann:
    if outcome[0] > outcome[1]:
        predict_ann_one.append(0)
    else:
        predict_ann_one.append(1)

In [103]:
truePositives = 0
trueNegatives = 0
falsePositives = 0
falseNegatives = 0

m = 0
for m in range(len(predict_ann_one)):
    if predict_ann_one[m] == y_test[m] and predict_ann_one[m] == 1:
        truePositives += 1
    elif predict_ann_one[m] == y_test[m] and predict_ann_one[m] == 0:
        trueNegatives += 1
    elif predict_ann_one[m] != y_test[m] and predict_ann_one[m] == 1:
        falsePositives += 1
    elif predict_ann_one[m] != y_test[m] and predict_ann_one[m] == 0:
        falseNegatives += 1

precisionPercent = truePositives*100/(truePositives+falsePositives)
print(str(precisionPercent) + "%")
recallPercent = truePositives*100/(truePositives+falseNegatives)
print(str(recallPercent) + "%")
fMeasure = 2*precisionPercent*recallPercent/(precisionPercent+recallPercent)
print(str(fMeasure) + "%")
accuracy = (truePositives+trueNegatives)*100/len(predict_ann_one)
print(str(accuracy) + "%")

64.96577473553205%
67.18146718146718%
66.05504587155963%
65.664%


In [104]:
results = ann.evaluate(X_test, np.array(y_test))

196/196 [==============================] - 2s 9ms/step - loss: 0.9567 - accuracy: 0.6566


In [105]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [106]:
cnn.fit(X_train, np.array(y_train), epochs=10)


Epoch 1/10
586/586 [==============================] - 25s 42ms/step - loss: 0.6017 - accuracy: 0.6663
Epoch 2/10
586/586 [==============================] - 20s 35ms/step - loss: 0.5050 - accuracy: 0.7557
Epoch 3/10
586/586 [==============================] - 18s 31ms/step - loss: 0.4563 - accuracy: 0.7835
Epoch 4/10
586/586 [==============================] - 20s 34ms/step - loss: 0.4174 - accuracy: 0.8061
Epoch 5/10
586/586 [==============================] - 19s 32ms/step - loss: 0.3818 - accuracy: 0.8282
Epoch 6/10
586/586 [==============================] - 17s 30ms/step - loss: 0.3374 - accuracy: 0.8531
Epoch 7/10
586/586 [==============================] - 18s 31ms/step - loss: 0.3025 - accuracy: 0.8669
Epoch 8/10
586/586 [==============================] - 18s 31ms/step - loss: 0.2502 - accuracy: 0.8957
Epoch 9/10
586/586 [==============================] - 18s 30ms/step - loss: 0.2027 - accuracy: 0.9175
Epoch 10/10
586/586 [==============================] - 18s 30ms/step - loss: 0.153

In [107]:
cnn_results = cnn.evaluate(X_test, np.array(y_test))

196/196 [==============================] - 2s 11ms/step - loss: 0.5988 - accuracy: 0.8003


In [108]:
predict_cnn = cnn.predict(X_test) 

In [109]:
predict_cnn_one = []
for outcome in predict_cnn:
    if outcome[0] > outcome[1]:
        predict_cnn_one.append(0)
    else:
        predict_cnn_one.append(1)

In [110]:
truePositives = 0
trueNegatives = 0
falsePositives = 0
falseNegatives = 0

m = 0
for m in range(len(predict_cnn_one)):
    if predict_cnn_one[m] == y_test[m] and predict_cnn_one[m] == 1:
        truePositives += 1
    elif predict_cnn_one[m] == y_test[m] and predict_cnn_one[m] == 0:
        trueNegatives += 1
    elif predict_cnn_one[m] != y_test[m] and predict_cnn_one[m] == 1:
        falsePositives += 1
    elif predict_cnn_one[m] != y_test[m] and predict_cnn_one[m] == 0:
        falseNegatives += 1

precisionPercent = truePositives*100/(truePositives+falsePositives)
print(str(precisionPercent) + "%")
recallPercent = truePositives*100/(truePositives+falseNegatives)
print(str(recallPercent) + "%")
fMeasure = 2*precisionPercent*recallPercent/(precisionPercent+recallPercent)
print(str(fMeasure) + "%")
accuracy = (truePositives+trueNegatives)*100/len(predict_cnn_one)
print(str(accuracy) + "%")

81.84931506849315%
76.8983268983269%
79.29661579296616%
80.032%


In [115]:
ann_2 = models.Sequential([
        layers.Flatten(input_shape=(32,32,3)),
        layers.Dense(100, activation='relu'),
        layers.Dense(2, activation='softmax')    
    ])

ann_2.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [116]:
ann_2.fit(np.array(X_train), np.array(y_train), epochs=10)

Epoch 1/10
586/586 [==============================] - 3s 4ms/step - loss: 0.7009 - accuracy: 0.6020
Epoch 2/10
586/586 [==============================] - 3s 4ms/step - loss: 0.5960 - accuracy: 0.6735
Epoch 3/10
586/586 [==============================] - 3s 5ms/step - loss: 0.5633 - accuracy: 0.7091
Epoch 4/10
586/586 [==============================] - 2s 4ms/step - loss: 0.5361 - accuracy: 0.7285
Epoch 5/10
586/586 [==============================] - 3s 5ms/step - loss: 0.5098 - accuracy: 0.7522
Epoch 6/10
586/586 [==============================] - 2s 4ms/step - loss: 0.4833 - accuracy: 0.7689
Epoch 7/10
586/586 [==============================] - 3s 5ms/step - loss: 0.4613 - accuracy: 0.7813
Epoch 8/10
586/586 [==============================] - 3s 5ms/step - loss: 0.4380 - accuracy: 0.7969
Epoch 9/10
586/586 [==============================] - 3s 5ms/step - loss: 0.4136 - accuracy: 0.8139
Epoch 10/10
586/586 [==============================] - 2s 4ms/step - loss: 0.3921 - accuracy: 0.8271

In [117]:
predict_ann_2 = ann_2.predict(X_test)
predict_ann_2_one = []
for outcome in predict_ann_2:
    if outcome[0] > outcome[1]:
        predict_ann_2_one.append(0)
    else:
        predict_ann_2_one.append(1)

In [118]:
truePositives = 0
trueNegatives = 0
falsePositives = 0
falseNegatives = 0

m = 0
for m in range(len(predict_ann_2_one)):
    if predict_ann_2_one[m] == y_test[m] and predict_ann_2_one[m] == 1:
        truePositives += 1
    elif predict_ann_2_one[m] == y_test[m] and predict_ann_2_one[m] == 0:
        trueNegatives += 1
    elif predict_ann_2_one[m] != y_test[m] and predict_ann_2_one[m] == 1:
        falsePositives += 1
    elif predict_ann_2_one[m] != y_test[m] and predict_ann_2_one[m] == 0:
        falseNegatives += 1

precisionPercent = truePositives*100/(truePositives+falsePositives)
print(str(precisionPercent) + "%")
recallPercent = truePositives*100/(truePositives+falseNegatives)
print(str(recallPercent) + "%")
fMeasure = 2*precisionPercent*recallPercent/(precisionPercent+recallPercent)
print(str(fMeasure) + "%")
accuracy = (truePositives+trueNegatives)*100/len(predict_ann_2_one)
print(str(accuracy) + "%")

65.01913265306122%
65.6048906048906%
65.31069827033951%
65.344%


In [119]:
results = ann_2.evaluate(X_test, np.array(y_test))

196/196 [==============================] - 1s 3ms/step - loss: 0.7479 - accuracy: 0.6534


In [122]:
cnn_5 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(300, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(10, activation='relu'),
    layers.Dense(2, activation='softmax') 
])
cnn_5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [123]:
cnn_5.fit(X_train, np.array(y_train), epochs=10)


Epoch 1/10
586/586 [==============================] - 39s 64ms/step - loss: 0.6136 - accuracy: 0.6572
Epoch 2/10
586/586 [==============================] - 37s 64ms/step - loss: 0.5107 - accuracy: 0.7485
Epoch 3/10
586/586 [==============================] - 39s 66ms/step - loss: 0.4545 - accuracy: 0.7857
Epoch 4/10
586/586 [==============================] - 38s 66ms/step - loss: 0.3941 - accuracy: 0.8208
Epoch 5/10
586/586 [==============================] - 38s 65ms/step - loss: 0.3316 - accuracy: 0.8530
Epoch 6/10
586/586 [==============================] - 39s 66ms/step - loss: 0.2459 - accuracy: 0.8960
Epoch 7/10
586/586 [==============================] - 38s 65ms/step - loss: 0.1539 - accuracy: 0.9396
Epoch 8/10
586/586 [==============================] - 38s 64ms/step - loss: 0.0929 - accuracy: 0.9638
Epoch 9/10
586/586 [==============================] - 38s 65ms/step - loss: 0.0671 - accuracy: 0.9764
Epoch 10/10
586/586 [==============================] - 39s 67ms/step - loss: 0.050

In [124]:
cnn_results = cnn_5.evaluate(X_test, np.array(y_test))

196/196 [==============================] - 4s 18ms/step - loss: 1.0302 - accuracy: 0.7789


In [127]:
predict_cnn_5 = cnn_5.predict(X_test) 

In [128]:
predict_cnn_5_one = []
for outcome in predict_cnn_5:
    if outcome[0] > outcome[1]:
        predict_cnn_5_one.append(0)
    else:
        predict_cnn_5_one.append(1)

In [129]:
truePositives = 0
trueNegatives = 0
falsePositives = 0
falseNegatives = 0

m = 0
for m in range(len(predict_cnn_5_one)):
    if predict_cnn_5_one[m] == y_test[m] and predict_cnn_5_one[m] == 1:
        truePositives += 1
    elif predict_cnn_5_one[m] == y_test[m] and predict_cnn_5_one[m] == 0:
        trueNegatives += 1
    elif predict_cnn_5_one[m] != y_test[m] and predict_cnn_5_one[m] == 1:
        falsePositives += 1
    elif predict_cnn_5_one[m] != y_test[m] and predict_cnn_5_one[m] == 0:
        falseNegatives += 1

precisionPercent = truePositives*100/(truePositives+falsePositives)
print(str(precisionPercent) + "%")
recallPercent = truePositives*100/(truePositives+falseNegatives)
print(str(recallPercent) + "%")
fMeasure = 2*precisionPercent*recallPercent/(precisionPercent+recallPercent)
print(str(fMeasure) + "%")
accuracy = (truePositives+trueNegatives)*100/len(predict_cnn_5_one)
print(str(accuracy) + "%")

75.76119402985074%
81.66023166023166%
78.60018581604211%
77.888%
